In [3]:
import json
import torch
from collections import defaultdict
from pathlib import Path

def build_flat_catalog(json_path: str, output_path: str):
    """
    Read a JSON catalog of lensing systems and write out a "flat" binary file
    with pre-stacked tensors for:
     - precomputed quantities (one row per system)
     - each mass-component type (rows of [sys_idx, *params]
     - each source-model type (rows of [sys_idx, *params])
    Also emits a small `param_map` per component/source type to decode columns.

    Args:
        json_path: path to the input JSON file (dict with key "SL_systems")
        output_path: path where to save the flat catalog (.pt)
    """
    # Load JSON
    with open(json_path, 'r') as f:
        catalog = json.load(f)
    systems = catalog.get("SL_systems", [])

    # Containers
    
    #For this one, differently form the following ones, there are not multiple tipes: we do not need default dict.
    #Also, here one row corresponds to one index, but for uniformity with following structures we write it like this.
    precomp_rows = {'sys_idxs': [], 'param_rows': []}
    
    labels={'sys_idxs': [], 'label_value': []}
    
    #default dict: mass_data["type"] does not exist beforehand, but this way it gets created authomatically when called
    mass_data = defaultdict(lambda: {'sys_idxs': [], 'param_rows': []})
    
    
    #This way we are able to deal eventually with multiple sources
    source_data = defaultdict(lambda: {'sys_idxs': [], 'param_rows': []})

    # Define param order maps
    
    
    precomp_map = ['D_l', 'D_s', 'D_ls', 'Theta_E']
    
#NOTE: These should be externalized.
    mass_param_map = {
        'SIS': ['pos_x','pos_y','redshift','vel_disp'],
        'NFW': ['pos_x', 'pos_y','mass_max','r_max_kpc','redshift'],
        'ExternalPotential': ['shear_x','shear_y','shear_strength','shear_angle_arcsec'],
        'PEMD': ['pos_x', 'pos_y', 'redshift', 'vel_disp', 'slope', 'orient', 'q']

    }
    
    
    
    source_param_map = {
        'Gaussian_blob': ['position_rad_x', 'position_rad_y', 'I' ,'orient_rad','q','std_kpc','redshift']
        # add other source types here
    }

    # Loop systems
    for sys in systems:
        i = sys['system_index']
        # precomputed
        pre = sys['precomputed']
        precomp_rows["sys_idxs"].append(i)
        
        row=[]
        for precomp_quant in precomp_map:
            row.append(pre[precomp_quant])
        precomp_rows["param_rows"].append(row)

        label=[0 if sys['lens_model']['num_substructures']==0 else 1]
        labels['sys_idxs'].append(i)
        labels['label_value'].append(label)

         # mass components
        for mc in sys['lens_model'].get('mass_components', []):
            mtype  = mc['type']
            params = mc['params']

            # 1) pull out all non-position params in order
            nonpos = []
            for key in mass_param_map[mtype]:
                if key not in ('pos_x', 'pos_y', 'shear_x', 'shear_y'):
                    nonpos.append(params[key][0] if isinstance(params[key], (list, tuple)) and len(params[key]) == 1 else params[key])



            # 2) build your final row as [pos_x, pos_y, *nonpos]
            # check if the key pos exists in the params dictionary
            if 'pos' in params:
                row = [params['pos'][0], params['pos'][1], *nonpos]

            elif 'shear_center' in params:
                row = [params['shear_center'][0], params['shear_center'][1], *nonpos]
            # store it
            mass_data[mtype]['sys_idxs'].append(i)
            mass_data[mtype]['param_rows'].append(row)

        # source models
        sm = sys['source_model']
        source_type = sm['type']
        params = sm['params']
       
            
        nonpos = []
        for key in source_param_map[source_type]:
            if key not in ('position_rad_x', 'position_rad_y'):
                nonpos.append(params[key][0] if isinstance(params[key], (list, tuple)) and len(params[key]) == 1 else params[key])

        # 2) build your final row as [pos_x, pos_y, *nonpos]
            
        row = [params['position_rad'][0], params['position_rad'][1], *nonpos]
            
        source_data[source_type]['sys_idxs'].append(i)
        source_data[source_type]['param_rows'].append(row)

    
    
    # Convert to tensors
    flat = {
        'precomputed': {},
        'mass_components': {},
        'source_models': {},
        'labels': {}
    }                   

    index_idxs=torch.tensor(labels['sys_idxs'], dtype=torch.long)
    index_vals=torch.tensor(labels['label_value'], dtype=torch.long)
    
    flat['labels'] = {
        'sys_idx': index_idxs,
        'label_values': index_vals
    }

    # Precomputed quantities — mirror the mass/source pattern
    pc_idxs = torch.tensor(precomp_rows['sys_idxs'], dtype=torch.long)
    pc_vals = torch.tensor(precomp_rows['param_rows'], dtype=torch.float32)
    


    
    flat['precomputed'] = {
        'sys_idx':   pc_idxs,
        'params':    pc_vals,
        'param_map': precomp_map
    }

    # Mass components
    for mtype, dat in mass_data.items():
        idxs = torch.tensor(dat['sys_idxs'], dtype=torch.long)
        rows = torch.tensor(dat['param_rows'], dtype=torch.float32)
        flat['mass_components'][mtype] = {
            'sys_idx':   idxs,
            'params':    rows,
            'param_map': mass_param_map[mtype]
        }

    # Source models
    for stype, dat in source_data.items():
        idxs = torch.tensor(dat['sys_idxs'], dtype=torch.long)
        rows = torch.tensor(dat['param_rows'], dtype=torch.float32)
        flat['source_models'][stype] = {
            'sys_idx':   idxs,
            'params':    rows,
            'param_map': source_param_map[stype]
        }

    # Save
    torch.save(flat, Path(output_path))
    print(f"Flat catalog written to {output_path}")




In [2]:
module('load', 'pytorch/gpu-cuda-12.1/2.2.0')


Loading pytorch/gpu-cuda-12.1/2.2.0
  Loading requirement: cuda/12.1 cudnn/8.9.2


In [4]:

json_path="../../catalogs/conor_train_gauss_source_10e8_6_resample_theta_val_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e8_6_resample_theta_val_2.pth"

build_flat_catalog(json_path, output_path)

json_path="../../catalogs/conor_train_gauss_source_10e9_resample_theta_train_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e9_resample_theta_train_2.pth"

build_flat_catalog(json_path, output_path)

json_path="../../catalogs/conor_train_gauss_source_10e9_resample_theta_val_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e9_resample_theta_val_2.pth"

build_flat_catalog(json_path, output_path)

json_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_train_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_train_2.pth"

build_flat_catalog(json_path, output_path)

json_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.pth"

build_flat_catalog(json_path, output_path)

json_path="../../catalogs/conor_train_gauss_source_10e11_resample_theta_train_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e11_resample_theta_train_2.pth"

build_flat_catalog(json_path, output_path)

json_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.pth"

build_flat_catalog(json_path, output_path)




Flat catalog written to ../../catalogs/conor_train_gauss_source_10e8_6_resample_theta_val_2.pth
Flat catalog written to ../../catalogs/conor_train_gauss_source_10e9_resample_theta_train_2.pth
Flat catalog written to ../../catalogs/conor_train_gauss_source_10e9_resample_theta_val_2.pth
Flat catalog written to ../../catalogs/conor_train_gauss_source_10e10_resample_theta_train_2.pth
Flat catalog written to ../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.pth
Flat catalog written to ../../catalogs/conor_train_gauss_source_10e11_resample_theta_train_2.pth
Flat catalog written to ../../catalogs/conor_train_gauss_source_10e11_resample_theta_val_2.pth


In [5]:
json_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.json"
output_path="../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.pth"

build_flat_catalog(json_path, output_path)


Flat catalog written to ../../catalogs/conor_train_gauss_source_10e10_resample_theta_val_2.pth


In [ ]:
module('load', 'pytorch/gpu-cuda-12.1/2.2.0')

In [ ]:

# Cell 1: imports & load
import torch
import pandas as pd
import matplotlib.pyplot as plt

# adjust this path as needed
catalog_path = "../../catalogs/test_small_no_sub_sis.pth"
flat = torch.load(catalog_path)


In [ ]:
# Cell 2: build precomputed DataFrame
pre = flat['precomputed']

print(pre.keys())
# params is a [N, K] tensor and param_map lists column names
params = pre['params'].numpy()
columns = pre['param_map']
df_pre = pd.DataFrame(params, columns=columns)
df_pre['sys_idx'] = pre['sys_idx'].numpy()



In [ ]:
labels= flat['labels']
print(labels.keys())

df_labels=pd.DataFrame( labels["label_values"])

print("Precomputed:")
display(df_labels.head())

df_labels.describe()



In [ ]:
# Cell 3: build one DataFrame per mass‐component type
df_mass = {}
for mtype, dat in flat['mass_components'].items():
    cols = dat['param_map']
    arr  = dat['params'].numpy()
    df    = pd.DataFrame(arr, columns=cols)
    df['sys_idx'] = dat['sys_idx'].numpy()
    df_mass[mtype] = df


In [ ]:
# Cell 4: build one DataFrame per source‐model type
df_src = {}
for stype, dat in flat['source_models'].items():
    cols = dat['param_map']
    arr  = dat['params'].numpy()
    df    = pd.DataFrame(arr, columns=cols)
    df['sys_idx'] = dat['sys_idx'].numpy()
    df_src[stype] = df


In [ ]:
# Cell 5: quick look at the tables
print("Precomputed:")
display(df_pre.head())

print("Mass types:", list(df_mass.keys()))
for mtype, df in df_mass.items():
    print(f"\n{mtype}")
    display(df.head())

print("Source types:", list(df_src.keys()))
for stype, df in df_src.items():
    print(f"\n{stype}")
    display(df.head())


In [ ]:
# # Cell 6: simple plots
# # 1) D_l vs Theta_E
# ax = df_pre.plot.scatter(x='D_l', y='Theta_E', title='D_l vs Θ_E', alpha=0.7)
# ax.grid(True)

# # 2) histogram of D_s
# plt.figure()
# df_pre['D_s'].plot.hist(bins=20, title='Distribution of D_s')
# plt.tight_layout()

# # 3) if NFW present, r_max_kpc vs mass_max
# if 'NFW' in df_mass:
#     ax2 = df_mass['NFW'].plot.scatter(
#         x='r_max_kpc', y='mass_max',
#         title='NFW: r_max_kpc vs mass_max', alpha=0.7
#     )
#     ax2.grid(True)

# plt.show()


In [ ]:
# import torch

# def pprint_structure(obj, indent=0):
#     pad = '  ' * indent
#     if isinstance(obj, dict):
#         for k, v in obj.items():
#             print(f"{pad}{k!r}: {type(v).__name__}", end='')
#             if hasattr(v, 'shape'):
#                 print(f"  shape={tuple(v.shape)}")
#             else:
#                 print()
#             pprint_structure(v, indent+1)
#     elif isinstance(obj, (list, tuple)):
#         for i, v in enumerate(obj):
#             print(f"{pad}[{i}]: {type(v).__name__}", end='')
#             if hasattr(v, 'shape'):
#                 print(f"  shape={tuple(v.shape)}")
#             else:
#                 print()
#             pprint_structure(v, indent+1)

# # Usage:
# catalog_path = "../../catalogs/small_example_catalog.pth"

# flat = torch.load(catalog_path)
# pprint_structure(flat)


In [ ]:
from lensing_system_broadcasting import LensingSystem
import torch

catalog_path = "../../catalogs/small_example_catalog.pth"
flat = torch.load(catalog_path)



masses_data = flat['mass_components']
for mtype, data in masses_data.items():
    # Convert to CUDA tensors
    masses_data[mtype]['params'] = data['params'].to(device='cuda:0')
    masses_data[mtype]['sys_idx'] = data['sys_idx'].to(device='cuda:0')


precomputed_data = flat['precomputed']

precomputed_data['params'] = precomputed_data['params'].to(device='cuda:0')
precomputed_data['sys_idx'] = precomputed_data['sys_idx'].to(device='cuda:0')


source_data = flat['source_models']
for stype, data in source_data.items():
    # Convert to CUDA tensors
    source_data[stype]['params'] = data['params'].to(device='cuda:0')
    source_data[stype]['sys_idx'] = data['sys_idx'].to(device='cuda:0')


lensing_system = LensingSystem(
    num_samples=100,
    masses_data=masses_data,
    precomputed_data=precomputed_data,
    source_data=source_data,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
)

In [ ]:
from shared_utils import _grid_lens
import matplotlib.pyplot as plt



grid = _grid_lens(6, 100, device="cuda")

images= lensing_system.forward(grid)[0]


for i in range(5):
    plt.imshow(images[i].cpu().detach().numpy(), cmap='gray')
    plt.show()


In [ ]:
# lets start by checking the mass classes
from lensing_system_broadcasting.lens_mass_components import NFW, PEMD


In [ ]:

# Cell 1: imports & load
import torch

# adjust this path as needed
catalog_path = "../../catalogs/small_example_catalog.pth"
flat = torch.load(catalog_path)

In [ ]:
Pemd_tensor_init=flat["mass_components"]["PEMD"]
print(Pemd_tensor_init.keys())



In [ ]:
Pemd_tensor_init=flat["mass_components"]["PEMD"]['params']

precomputed_tensor_init=flat["precomputed"]
print(precomputed_tensor_init["params"].device)
precomputed_tensor_init["params"] = precomputed_tensor_init["params"].to("cuda")

print(precomputed_tensor_init.keys())


In [ ]:
my_pemd=PEMD(Pemd_tensor_init.to("cuda:0"),
            precomputed_tensor_init)

In [ ]:
Nfw_tensor_init=flat["mass_components"]["NFW"]['params']

#now the initialization of the precomputed quantities is more tricky
Nfw_indexes=flat["mass_components"]["NFW"]['sys_idx']



print(torch.max(Nfw_indexes))

print(len(Nfw_tensor_init))



#now we slice up the nfw tensor to get the pre
precomputed_nfw_params=flat["precomputed"]["params"][Nfw_indexes]
precomputed_nfw_map=flat["precomputed"]["param_map"]

precomputed_init_nfw={
    "params":precomputed_nfw_params.to("cuda:0"),
    "param_map":precomputed_nfw_map
}
print(Nfw_tensor_init.type())

In [ ]:
device = "cuda:0"
dtype  = torch.float32

# assume `Nfw_tensor_init` is already your [B,7] tensor including the D_l, D_s, D_ls columns
#param_tensor = Nfw_tensor_init.to(device=device, dtype=dtype)

my_nfw = NFW(Nfw_tensor_init.to(device=device, dtype=dtype),
             precomputed_init_nfw,
             device=device, dtype=dtype)


In [ ]:
from lensing_system_broadcasting import LensModel
# Cell 1: imports & load
import torch

# adjust this path as needed
catalog_path = "../../catalogs/small_example_catalog.pth"
flat = torch.load(catalog_path)


In [ ]:
masses_data = flat['mass_components']
for mtype, data in masses_data.items():
    # Convert to CUDA tensors
    masses_data[mtype]['params'] = data['params'].to(device='cuda:0')
    masses_data[mtype]['sys_idx'] = data['sys_idx'].to(device='cuda:0')


precomputed_data = flat['precomputed']

precomputed_data['params'] = precomputed_data['params'].to(device='cuda:0')
precomputed_data['sys_idx'] = precomputed_data['sys_idx'].to(device='cuda:0')

my_lens_model = LensModel(
    number_of_systems=100,
    masses_data=masses_data,
    precomputed=precomputed_data,
)

# Lets try a forward pass

In [ ]:
from shared_utils import _grid_lens

my_grid= _grid_lens(3, 500, device="cuda:0")

In [ ]:
deflectedPoints=my_lens_model(my_grid)

In [ ]:
print(deflectedPoints.shape)

In [ ]:
first_field=deflectedPoints[5]


print(first_field)